## Setup

### Import libraries

In [1]:
from benchmarks.experiments.wrapper_overhead.common.deployment_utility import WrapperOverheadDeploymentUtility

### Setup Common Resources

In [2]:
region = 'us-east-2'
deployment_utility: WrapperOverheadDeploymentUtility = WrapperOverheadDeploymentUtility(region)

## Deploy Associated AWS functions

### DNA Visualization

#### Boto3 Direct Calls

In [7]:
direct_calls_dir = "./dna_visualization/external_database/boto3_direct" # Boto3 only direct calls
deployment_utility.deploy_experiment(direct_calls_dir)

Deploying boto3_direct workload
wo-dna_vis-ed-direct

Deploying wo-dna_vis-ed-direct-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.82kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 0.603 Collecting dna_features_viewer
#7 0.682   Downloading dna_features_viewer-3.1.3-py3-none-any.whl (31 kB)
#7 1.143 Collecting boto3==1.34.43
#7 1.158   Downloading boto3-1.34.43-py3-none-any.whl (139 kB)
#7 1.173      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.6 MB/s eta 0:00:00
#7 1.230 Collecting pyyaml==6.0.1
#7 1.243   Download

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize]
85cd74e8c31c: Preparing
678c63256b18: Preparing
8910df81c1f9: Preparing
a317458fc3e5: Preparing
48ab2ac5e1a5: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
61728103c778: Waiting
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
11c57b9ec508: Waiting
25a97811023e: Preparing
eb4ec3a6ddd2: Waiting
59e530c8b37a: Waiting
71786097e499: Waiting
25a97811023e: Waiting
48ab2ac5e1a5: Layer already exists
61728103c778: Layer already exists
11c57b9ec508: Layer already exists
59e530c8b37a: Layer already exists
8910df81c1f9: Pushed
678c63256b18: Pushed
85cd74e8c31c: Pushed
eb4ec3a6ddd2: Layer already exists
71786097e499: Layer already exists
25a97811023e: Layer already exists
a317458fc3e5: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-visualize:latest to ECR.


latest: digest: sha256:21d3c36ecc5074289a8ae949401d871e76fb9bb8c8b3a3697026fc26201263a9 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-visualize

Deploying wo-dna_vis-ed-direct-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-direct-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context:
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.69kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 90.50 Collecting boto3==1.34.43
#7 90.57   Downloading boto3-1.34.43-py3-none-any.whl (139 kB)
#7 90.60      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.9 MB/s eta 0:00:00
#7 90.66 Collecting pyyaml==6.0.1
#7 90.67   Downloading PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (736 kB)
#7 90.

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input]
11cd988a64f0: Preparing
cfe0867d9ee0: Preparing
452bae9784c8: Preparing
2404e2fda336: Preparing
16a2a762583c: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
61728103c778: Waiting
59e530c8b37a: Waiting
11c57b9ec508: Waiting
eb4ec3a6ddd2: Waiting
25a97811023e: Waiting
71786097e499: Waiting
16a2a762583c: Layer already exists
61728103c778: Layer already exists
11c57b9ec508: Layer already exists
59e530c8b37a: Layer already exists
452bae9784c8: Pushed
11cd988a64f0: Pushed
cfe0867d9ee0: Pushed
eb4ec3a6ddd2: Layer already exists
71786097e499: Layer already exists
25a97811023e: Layer already exists
2404e2fda336: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-direct-get_input:latest to ECR.


latest: digest: sha256:51c3356e432b8b9603b19861d246045979b2e0363fef514986d74218caa35210 size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-direct-get_input
Completed deployment of wo-dna_vis-ed-direct




#### Boto3 SNS calls

In [4]:
sns_calls_dir = "./dna_visualization/external_database/boto3_sns" # Boto3 only sns calls
deployment_utility.deploy_experiment(sns_calls_dir)

Found credentials in environment variables.


Deploying boto3_sns workload
wo-dna_vis-ed-sns

Deploying wo-dna_vis-ed-sns-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-sns-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 10.5s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 108.89kB done
#4 DONE 0.0s

#5 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#5 resolve public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1 0.0s done
#5 extracting sha256:acf752993f144e5f0f83c87c268ae7fbd884593d3d32d1abda48e267bd2be281
#5 extracting sha256:acf752993f144e5f0f83c87c268ae7fbd884593d3d32d1abda48e267bd2be281 done
#5 sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1 772B / 772B done
#5 sha256:7b442e7e2e42407112b0064181b07c6c70c56411721e213adb06740f56d692f9 1.58

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-sns-visualize]
54f80a2d663d: Preparing
d3dd124a674e: Preparing
4feecb1756fc: Preparing
330a4d8ec789: Preparing
48ab2ac5e1a5: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
11c57b9ec508: Waiting
59e530c8b37a: Waiting
eb4ec3a6ddd2: Waiting
71786097e499: Waiting
25a97811023e: Waiting
61728103c778: Waiting
48ab2ac5e1a5: Pushed
4feecb1756fc: Pushed
d3dd124a674e: Pushed
54f80a2d663d: Pushed
eb4ec3a6ddd2: Pushed
59e530c8b37a: Pushed
71786097e499: Pushed
61728103c778: Pushed
330a4d8ec789: Pushed
11c57b9ec508: Pushed
25a97811023e: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-sns-visualize:latest to ECR.


latest: digest: sha256:5e2e43b34ab0ef1b924ec40db4dcc1f1beda1accbb0ce09517ebbf54797df063 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-sns-visualize
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-sns-visualize-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-sns-visualize-sns_topic:8edc03de-56e9-4677-87b4-57caf0504356

Deploying wo-dna_vis-ed-sns-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-sns-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 107.14kB done
#4 DONE 0.0s

#5 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#5 CACHED

#6 [2/6] COPY requirements.txt ./
#6 DONE 0.0s

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 0.921 Collecting boto3==1.34.43
#7 0.995   Downloading boto3-1.34.43-py3-none-any.whl (139 kB)
#7 1.015      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.8 MB/s eta 0:00:00
#7 1.068 Collecting pyyaml==6.0.1
#7 1.078   Downloading PyYAML-6.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (736 kB)
#7 1.097      ━━━━━━━━━━━━━━━━━

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-sns-get_input]
7186a377c4fb: Preparing
ce2bac13af20: Preparing
3252edd72986: Preparing
348f9a7ff89e: Preparing
16a2a762583c: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
11c57b9ec508: Waiting
eb4ec3a6ddd2: Waiting
71786097e499: Waiting
59e530c8b37a: Waiting
61728103c778: Waiting
25a97811023e: Waiting
ce2bac13af20: Pushed
3252edd72986: Pushed
16a2a762583c: Pushed
7186a377c4fb: Pushed
348f9a7ff89e: Pushed
eb4ec3a6ddd2: Pushed
71786097e499: Pushed
59e530c8b37a: Pushed
61728103c778: Pushed
11c57b9ec508: Pushed
25a97811023e: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-sns-get_input:latest to ECR.


latest: digest: sha256:b746f59a08d8bba76673ddae90e29c0ea811a969ccf0081359538cf46da330fa size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-sns-get_input
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-sns-get_input-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-dna_vis-ed-sns-get_input-sns_topic:efc8b0a0-add0-45a3-9f62-458d6aa149d4
Completed deployment of wo-dna_vis-ed-sns




#### AWS Step Function

In [24]:
aws_step_functions_dir = "./dna_visualization/external_database/aws_step_function" # AWS Step Functions
deployment_utility.deploy_experiment(aws_step_functions_dir)

Deploying aws_step_function workload
wo-dna_vis-ed-step_function

Deploying wo-dna_vis-ed-step_function-visualize
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-step_function-visualize-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 108.89kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:5cfa003e5b36adb2362095ca91d8275ab13197d8337f117f0ec11bfab6959b5c done
#11 naming to docker.io/library/wo-dna_vis-ed-ste

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-visualize]
9df28a6a8d0d: Preparing
a8f9a66cb326: Preparing
c1e1781d79d8: Preparing
8e1b406f394f: Preparing
3a89be36bfcd: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
05788b7af5ed: Waiting
706e59713fa4: Waiting
6849347e354b: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
0a20c8da2344: Waiting
3a89be36bfcd: Layer already exists
8e1b406f394f: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
706e59713fa4: Layer already exists
052242efe275: Layer already exists
6849347e354b: Layer already exists
274a79fec6a9: Layer already exists
a8f9a66cb326: Pushed
c1e1781d79d8: Pushed
9df28a6a8d0d: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-visualize:latest to ECR.


latest: digest: sha256:781efce4e440e1ecaac77033f6960c4cf6b747b6230d89ce49078a46a17766e4 size: 2624
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-step_function-visualize

Deploying wo-dna_vis-ed-step_function-get_input
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-dna_vis-ed-step_function-get_input-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 325B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:9ae4af1397c3c7ac09aca53cca2621c9fdd5f0a2674f2bf05920888d24883f91
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.27kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:fbbd28bd98bffefaee60116f5bb26663bf76da7c8e809a63dc0cf1b8d0654aed done
#11 naming to docker.io/library/wo-dna_vis-ed-ste

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-get_input]
edbf480faeb2: Preparing
e41be327bb84: Preparing
52c0086b9e60: Preparing
18f3008705b6: Preparing
1c58f3f28eec: Preparing
0a20c8da2344: Preparing
05788b7af5ed: Preparing
706e59713fa4: Preparing
052242efe275: Preparing
274a79fec6a9: Preparing
6849347e354b: Preparing
706e59713fa4: Waiting
052242efe275: Waiting
274a79fec6a9: Waiting
6849347e354b: Waiting
0a20c8da2344: Waiting
05788b7af5ed: Waiting
18f3008705b6: Layer already exists
1c58f3f28eec: Layer already exists
0a20c8da2344: Layer already exists
05788b7af5ed: Layer already exists
052242efe275: Layer already exists
706e59713fa4: Layer already exists
274a79fec6a9: Layer already exists
6849347e354b: Layer already exists
52c0086b9e60: Pushed
e41be327bb84: Pushed
edbf480faeb2: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-step_function-get_input:latest to ECR.


latest: digest: sha256:5cf9292cc0cbf3c6ba258fa8eb229c57739a4a167a9d61dc4ea9c6617d9b486e size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-dna_vis-ed-step_function-get_input
Completed deployment of wo-dna_vis-ed-step_function


Delete role arn
Old resources for Removed
Creating state machine iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/state_machine-wo-dna_vis-ed-step_function-policy
Updating state machine
Resulting State Machine ARN: arn:aws:states:us-east-2:485595969306:stateMachine:state_machine-wo-dna_vis-ed-step_function
Completed deployment of wo-dna_vis-ed-step_function




True

#### Multi-X

In [3]:
multi_x_dir = "./dna_visualization/external_database/multi_x" # Multi_x
deployment_utility.deploy_experiment(multi_x_dir)

Found credentials in environment variables.
Found credentials in environment variables.


Deploying multi_x workload
wo-dna_vis-ed-multi_x-0_0_2


Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.


Removed workflow wo-dna_vis-ed-multi_x-0_0_2


Deploying workflow wo-dna_vis-ed-multi_x with version 0.0.2
Found credentials in environment variables.
Building deployment package
Building deployment plan
Found credentials in environment variables.
Found credentials in environment variables.
Executing deployment plan
Found credentials in environment variables.
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 326B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.11
#2 DONE 0.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.11@sha256:e330bd08eb6e689d399d275bcd4c41e024dabb1d6917fb8a2bf06ae4e0a22876
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 109.89kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 

...

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-multi_x-0_0_2-visualize_aws-us-east-2]
c2135100defa: Preparing
163866c5f3b9: Preparing
e922c0b5dce3: Preparing
c478a1ff5225: Preparing
1116e88cf31a: Preparing
73985012f706: Preparing
206ba18557d7: Preparing
706e59713fa4: Preparing
4acdd4f61051: Preparing
9c79f1260ae3: Preparing
6849347e354b: Preparing
73985012f706: Waiting
206ba18557d7: Waiting
706e59713fa4: Waiting
4acdd4f61051: Waiting
9c79f1260ae3: Waiting
6849347e354b: Waiting
e922c0b5dce3: Pushed
1116e88cf31a: Pushed
163866c5f3b9: Pushed
c2135100defa: Pushed
4acdd4f61051: Pushed
73985012f706: Pushed
706e59713fa4: Pushed
9c79f1260ae3: Pushed
c478a1ff5225: Pushed
206ba18557d7: Pushed
6849347e354b: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-dna_vis-ed-multi_x-0_0_2-visualize_aws-us-east-2:latest to ECR.


latest: digest: sha256:0092e224bf908abdefcf3f327f99a47e4768e61abe1161a73ee50e585e65bc08 size: 2622


Uploading workflow to configuration server
Found credentials in environment variables.
Workflow wo-dna_vis-ed-multi_x with version 0.0.2 deployed
Workflow id: wo-dna_vis-ed-multi_x-0.0.2


TypeError: 'Config' object is not subscriptable

### Image Processing

#### Boto3 Direct Calls

In [11]:
direct_calls_dir = "./image_processing/external_database/boto3_direct" # Boto3 only direct calls
deployment_utility.deploy_experiment(direct_calls_dir)

Deploying boto3_direct workload
wo-im_p-ed-direct

Deploying wo-im_p-ed-direct-filter
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-direct-filter-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 322B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.3s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.92kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 DONE 0.0s

#9 [5/6] COPY src ./src
#9 DONE 0.0s

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 DONE 0.0s

#11 exporting to image
#11 exporting layers 0.0s done
#11 writing image sha256:64edc716bfa95affc97bd7b6c3b45939a227963ca4d2bbf2fc4b38e952682fad done
#11 naming to docker.io/library/wo-im_p-ed-direct

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-direct-filter]
f1934069ce95: Preparing
71556d85287d: Preparing
ce601cf2d588: Preparing
f98c2af0d2a4: Preparing
d77812e07c33: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
11c57b9ec508: Waiting
59e530c8b37a: Waiting
eb4ec3a6ddd2: Waiting
71786097e499: Waiting
25a97811023e: Waiting
61728103c778: Waiting
d77812e07c33: Layer already exists
f98c2af0d2a4: Layer already exists
61728103c778: Layer already exists
11c57b9ec508: Layer already exists
eb4ec3a6ddd2: Layer already exists
59e530c8b37a: Layer already exists
71786097e499: Layer already exists
25a97811023e: Layer already exists
71556d85287d: Pushed
ce601cf2d588: Pushed
f1934069ce95: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-direct-filter:latest to ECR.


latest: digest: sha256:0c4937c3484e83a8bcdb827f82f65078dd1a7f24b9d6b01b773074fa8b9969df size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-direct-filter

Deploying wo-im_p-ed-direct-grayscale
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-direct-grayscale-policy
Creating lambda function


...

latest: digest: sha256:1f00c98a9fa520ad7c1c6f0731ae194dd0fffb2cb0184f5b521438a58812b98e size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-direct-rotate

Deploying wo-im_p-ed-direct-flip
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-direct-flip-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 320B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 106.94kB done
#5 DONE 0.0s

#6 [4/6] COPY app.py ./
#6 CACHED

#7 [5/6] COPY src ./src
#7 CACHED

#8 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [2/6] COPY requirements.txt ./
#9 CACHED

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 CACHED

#11 exporting to image
#11 exporting layers done
#11 writing image sha256:9e75c14856aa24f0e3cf0a742d8f397435258abdd47844d87b6eaa64d1074a97 done
#11 naming to docker.io/library/wo-im_p-ed-direct-flip:latest d

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-direct-flip]
ea1b285ec1c1: Preparing
f5a193e42914: Preparing
2c9bb110f1ad: Preparing
f98c2af0d2a4: Preparing
d77812e07c33: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
11c57b9ec508: Waiting
eb4ec3a6ddd2: Waiting
25a97811023e: Waiting
71786097e499: Waiting
59e530c8b37a: Waiting
61728103c778: Waiting
f5a193e42914: Layer already exists
ea1b285ec1c1: Layer already exists
d77812e07c33: Layer already exists
2c9bb110f1ad: Layer already exists
f98c2af0d2a4: Layer already exists
61728103c778: Layer already exists
11c57b9ec508: Layer already exists
71786097e499: Layer already exists
eb4ec3a6ddd2: Layer already exists
59e530c8b37a: Layer already exists
25a97811023e: Layer already exists


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-direct-flip:latest to ECR.


latest: digest: sha256:ee49f42f95fec07b180a998d92d81a168eab54827abf2b68475cc3a315b7e2d2 size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-direct-flip
Completed deployment of wo-im_p-ed-direct




#### Boto3 SNS calls

In [6]:
sns_calls_dir = "./image_processing/external_database/boto3_sns" # Boto3 only sns calls
deployment_utility.deploy_experiment(sns_calls_dir)

Deploying boto3_sns workload
wo-im_p-ed-sns

Deploying wo-im_p-ed-sns-filter
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-sns-filter-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 322B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.4s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 107.49kB done
#5 DONE 0.0s

#6 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#6 CACHED

#7 [2/6] COPY requirements.txt ./
#7 CACHED

#8 [4/6] COPY app.py ./
#8 CACHED

#9 [5/6] COPY src ./src
#9 CACHED

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 CACHED

#11 exporting to image
#11 exporting layers done
#11 writing image sha256:5b53e4bd42fc5bf8be8cd356fd00371ff3456faa37cc1ab1b4a2e8081ccdb3ad done
#11 naming to docker.io/library/wo-im_p-ed-sns-filter:latest do

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-sns-filter]
5f8addb365a3: Preparing
bca2f90c4a3b: Preparing
77723512e2e2: Preparing
f98c2af0d2a4: Preparing
d77812e07c33: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
11c57b9ec508: Waiting
59e530c8b37a: Waiting
eb4ec3a6ddd2: Waiting
25a97811023e: Waiting
71786097e499: Waiting
61728103c778: Waiting
bca2f90c4a3b: Layer already exists
5f8addb365a3: Layer already exists
d77812e07c33: Layer already exists
77723512e2e2: Layer already exists
f98c2af0d2a4: Layer already exists
61728103c778: Layer already exists
11c57b9ec508: Layer already exists
71786097e499: Layer already exists
59e530c8b37a: Layer already exists
eb4ec3a6ddd2: Layer already exists
25a97811023e: Layer already exists


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-sns-filter:latest to ECR.


latest: digest: sha256:63f93179f6df11baf6654c1c9df589c2477a2bb29865799aecb2c900e46408a5 size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-sns-filter
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-filter-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-filter-sns_topic:4c7c6ed2-461e-4ec0-94fb-81b7e1c2dbb3

Deploying wo-im_p-ed-sns-grayscale
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-sns-grayscale-policy
Creating lambda function


...

latest: digest: sha256:f4c37f506f16e81e31a97ad9aadc668eeae2d74761aa337e875d242cbdb9a051 size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-sns-rotate
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-rotate-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-rotate-sns_topic:0cca95b0-eb6e-4575-a4d3-247996ca002c

Deploying wo-im_p-ed-sns-flip
Removing old resources
Old resources for Removed
Creating iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/wo-im_p-ed-sns-flip-policy
Creating lambda function


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 320B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.8
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.8@sha256:1c82a47e73abcf7948aeaa3c39503f6d2306351ec3c8096b4307fa1fbf6bbcb1
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 107.51kB done
#5 DONE 0.0s

#6 [5/6] COPY src ./src
#6 CACHED

#7 [3/6] RUN pip3 install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [4/6] COPY app.py ./
#8 CACHED

#9 [2/6] COPY requirements.txt ./
#9 CACHED

#10 [6/6] COPY multi_x_serverless ./multi_x_serverless
#10 CACHED

#11 exporting to image
#11 exporting layers done
#11 writing image sha256:5ea5ba427ae8be0913bc148a51b43e20c6aa916ecf50c41b739dfe76c71a52dc done
#11 naming to docker.io/library/wo-im_p-ed-sns-flip:latest done

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-sns-flip]
606c48cd5ad0: Preparing
e82765099757: Preparing
6c960fdd7f33: Preparing
f98c2af0d2a4: Preparing
d77812e07c33: Preparing
61728103c778: Preparing
11c57b9ec508: Preparing
59e530c8b37a: Preparing
eb4ec3a6ddd2: Preparing
71786097e499: Preparing
25a97811023e: Preparing
61728103c778: Waiting
11c57b9ec508: Waiting
59e530c8b37a: Waiting
eb4ec3a6ddd2: Waiting
71786097e499: Waiting
25a97811023e: Waiting
606c48cd5ad0: Layer already exists
d77812e07c33: Layer already exists
f98c2af0d2a4: Layer already exists
6c960fdd7f33: Layer already exists
e82765099757: Layer already exists
61728103c778: Layer already exists
59e530c8b37a: Layer already exists
11c57b9ec508: Layer already exists
71786097e499: Layer already exists
eb4ec3a6ddd2: Layer already exists
25a97811023e: Layer already exists


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-sns-flip:latest to ECR.


latest: digest: sha256:cbbec37d82eda7fbc89c9e9a9e73e8b84a6759533ece455e156cd06e7e2019fd size: 2623
Resulting Function ARN: arn:aws:lambda:us-east-2:485595969306:function:wo-im_p-ed-sns-flip
Creating sns topic and subscribing lambda function
Resulting Topic ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-flip-sns_topic
Resulting Subscription ARN: arn:aws:sns:us-east-2:485595969306:wo-im_p-ed-sns-flip-sns_topic:a5fea16f-f72b-430c-90c9-448e52140e0e
Completed deployment of wo-im_p-ed-sns




#### AWS Step Function

In [3]:
aws_step_functions_dir = "./image_processing/external_database/aws_step_function" # AWS Step Functions
deployment_utility.deploy_experiment(aws_step_functions_dir)

Found credentials in environment variables.


Found credentials in environment variables.


Delete role arn
Old resources for Removed
Creating state machine iam role
Resulting Policy ARN: arn:aws:iam::485595969306:role/state_machine-wo-im_p-ed-step_function-policy
Creating state machine
Resulting State Machine ARN: arn:aws:states:us-east-2:485595969306:stateMachine:state_machine-wo-im_p-ed-step_function
Completed deployment of wo-im_p-ed-step_function




#### Multi-X

In [3]:
multi_x_dir = "./image_processing/external_database/multi_x" # Multi_x
deployment_utility.deploy_experiment(multi_x_dir)

Found credentials in environment variables.
Found credentials in environment variables.


Deploying multi_x workload
wo-im_p-ed-multi_x-0.0.1


Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Deploying workflow wo-im_p-ed-multi_x with version 0.0.1
Found credentials in environment variables.


Removed workflow wo-im_p-ed-multi_x-0.0.1


Building deployment package
Building deployment plan
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Found credentials in environment variables.
Executing deployment plan
Found credentials in environment variables.
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 326B done
#1 DONE 0.0s

#2 [internal] load metadata for public.ecr.aws/lambda/python:3.11
#2 DONE 0.3s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/6] FROM public.ecr.aws/lambda/python:3.11@sha256:df7d15216f48174e42b5fb07ae194a233430a07b62bbce87790e7c4a458e60bf
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 113.09kB done
#5 DONE 0.0s

#6 [2/6] COPY requirements.txt ./
#6 CACHED

#7 [3/6] RUN pip3 install -

...

Login Succeeded
The push refers to repository [485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-multi_x-0_0_1-resize_aws-us-east-2]
9c89cc5151e7: Preparing
3bdf11507bf2: Preparing
6d9bc3d33d24: Preparing
de7915fda29b: Preparing
5c8fb27b0619: Preparing
8cc4aea68aa7: Preparing
9e3c8bb99d58: Preparing
59e530c8b37a: Preparing
ccc8d8710e4d: Preparing
107475a7540d: Preparing
25a97811023e: Preparing
9e3c8bb99d58: Waiting
59e530c8b37a: Waiting
ccc8d8710e4d: Waiting
107475a7540d: Waiting
25a97811023e: Waiting
8cc4aea68aa7: Waiting
6d9bc3d33d24: Pushed
5c8fb27b0619: Pushed
3bdf11507bf2: Pushed
9c89cc5151e7: Pushed
8cc4aea68aa7: Pushed
ccc8d8710e4d: Pushed
de7915fda29b: Pushed
59e530c8b37a: Pushed
107475a7540d: Pushed
9e3c8bb99d58: Pushed
25a97811023e: Pushed


Successfully pushed Docker image 485595969306.dkr.ecr.us-east-2.amazonaws.com/wo-im_p-ed-multi_x-0_0_1-resize_aws-us-east-2:latest to ECR.


latest: digest: sha256:7e24a00c82ac852c026e7171a080d6eaca491069f5fbad0b5f88cf2c147d8de2 size: 2621


Uploading workflow to configuration server
Found credentials in environment variables.
Workflow wo-im_p-ed-multi_x with version 0.0.1 deployed
Workflow id: wo-im_p-ed-multi_x-0.0.1


Completed deployment of wo-im_p-ed-multi_x-0.0.1


